In [2]:
import pandas as pd
import numpy as np
# import tensorflow as tf
# import tensorboard as tb
# tf.io.gfile = tb.compat.tensorflow_stub.io.gfile
from tensorboardX import SummaryWriter

import os
from tqdm import tqdm

In [3]:
data_dir = '2022-06-02'

In [4]:
# 2020 papers
pmc_metadata = pd.read_csv(f'valid_metadata_{data_dir}.csv',sep='\t')
pmc_metadata['publish_time'] = pd.to_datetime(pmc_metadata['publish_time'],format="%Y-%m-%d")

In [5]:
# citation net
citation_net = pd.read_csv(f'citation_net_{data_dir}.tsv',sep='\t')

In [6]:
needed_uid = set(citation_net["citing_uid"]) | set(citation_net["cited_uid"]) | set(pmc_metadata["cord_uid"])

In [7]:
len(needed_uid)

267949

In [8]:
# paper metadata
all_metadata = pd.read_csv(f"../{data_dir}/metadata.csv",low_memory=False)
all_metadata = all_metadata.drop_duplicates('cord_uid',keep='first')
all_metadata = all_metadata.set_index('cord_uid')

In [9]:
# paper needed
needed_metadata = all_metadata.loc[all_metadata.index.isin(needed_uid)]

In [10]:
needed_metadata.shape

(267948, 18)

In [11]:
needed_metadata = needed_metadata[['title','abstract']]

In [12]:
needed_metadata.fillna('',inplace=True)

In [13]:
needed_metadata.to_csv(f'needed_paper_metadata_{data_dir}.csv')

In [14]:
needed_metadata

,title,abstract
cord_uid,,
bufbjdmw,Interstitial lung diseases in children,Interstitial lung disease (ILD) in infants and...
a564l6vs,Database resources of the National Center for ...,The National Center for Biotechnology Informat...
y86m85pe,Public health round-up,
6fw4thkq,Impaired inflammasome activation and bacterial...,Glucose-6-phosphate dehydrogenase (G6PD) is th...
zaeg1ujv,Recombinant Rotaviruses Rescued by Reverse Gen...,Rotavirus (RV) replicates in round-shaped cyto...
...,...,...
3b63pnxq,2022 ACC Expert Consensus Decision Pathway on ...,
3x1mm80o,“Chronic granulomatous invasive fungal rhinosi...,INTRODUCTION AND IMPORTANCE: Granulomatous chr...
aeuy92bx,Potent neutralization of 2019 novel coronaviru...,"2019-nCoV, which is a novel coronavirus emerge..."


In [16]:
needed_metadata = needed_metadata.set_index('cord_uid')

,title,abstract
cord_uid,,
bufbjdmw,Interstitial lung diseases in children,Interstitial lung disease (ILD) in infants and...
a564l6vs,Database resources of the National Center for ...,The National Center for Biotechnology Informat...
y86m85pe,Public health round-up,NaN
6fw4thkq,Impaired inflammasome activation and bacterial...,Glucose-6-phosphate dehydrogenase (G6PD) is th...
zaeg1ujv,Recombinant Rotaviruses Rescued by Reverse Gen...,Rotavirus (RV) replicates in round-shaped cyto...
...,...,...
3b63pnxq,2022 ACC Expert Consensus Decision Pathway on ...,NaN
3x1mm80o,“Chronic granulomatous invasive fungal rhinosi...,INTRODUCTION AND IMPORTANCE: Granulomatous chr...
aeuy92bx,Potent neutralization of 2019 novel coronaviru...,"2019-nCoV, which is a novel coronavirus emerge..."


In [13]:
%run scibert.ipynb

In [14]:
tokenizer,model = scibert()

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(31090, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [16]:
title_dataset = texts(needed_metadata['title'].tolist())

In [17]:
needed_metadata['title']

cord_uid
a564l6vs    Database resources of the National Center for ...
dxdr6oan    Generation of protective pneumococcal-specific...
iwvjz3bf    Targeting the IL-22/IL-22BP axis enhances tigh...
d1pd09zj    Synthetic carbohydrate-based vaccines: challen...
44ip3jda    Identification of antigens presented by MHC fo...
                                  ...                        
ir1s5yfp    Impact of COVID‐19 on dental education: How co...
9m3efjpx    Improved Glycemic Control With a Digital Healt...
tlxews76    Novel therapeutics for the treatment of hypert...
kg1daxuu    The impact of policy mixes on new energy vehic...
f0t6n2j1    An initial report from the French SOT COVID Re...
Name: title, Length: 84391, dtype: object

In [18]:
title_dataloader = texts_dataloader(title_dataset,batch_size=64,tokenizer=tokenizer)

In [19]:
title_embs = []
with torch.no_grad():
    model.cuda()
    model.eval()
    for step,(input_ids,attention_mask,token_type_ids) in tqdm(enumerate(title_dataloader)):
        output = model(input_ids=input_ids.cuda(), attention_mask=attention_mask.cuda(), token_type_ids=token_type_ids.cuda())
#         print(output.shape)
        titles_emb = output.last_hidden_state.mean(dim=1).detach()
#         print(titles_emb.shape)
        title_embs.append(titles_emb.cpu().numpy())
#         print(titles_emb.flatten().cpu().numpy().shape)
#         break
#         break

1319it [08:40,  2.53it/s]


In [20]:
title_embs_array = np.vstack(title_embs)
title_embs_df = pd.DataFrame(data=title_embs_array,columns=['emb'+str(i) for i in range(768)],index=needed_metadata.index)

In [21]:
title_embs_df.shape

(84391, 768)

In [22]:
abstract_dataset = texts(needed_metadata['abstract'].tolist())
abstract_dataloader = texts_dataloader(abstract_dataset,batch_size=4,tokenizer=tokenizer)

In [23]:
abstract_embs = []
with torch.no_grad():
    model.cuda()
    model.eval()
    for step,(input_ids,attention_mask,token_type_ids) in tqdm(enumerate(abstract_dataloader)):
        output = model(input_ids=input_ids.cuda(), attention_mask=attention_mask.cuda(), token_type_ids=token_type_ids.cuda())
        abstract_emb = output.last_hidden_state.mean(dim=1).detach()
        abstract_embs.append(abstract_emb.cpu().numpy())

21098it [2:08:57,  2.73it/s]


In [27]:
abstract_embs_array = np.vstack(abstract_embs)
abstract_embs_df = pd.DataFrame(data=abstract_embs_array,columns=['emb'+str(i) for i in range(768)],index=needed_metadata.index)

In [28]:
abstract_embs_df.shape

(84391, 768)

In [25]:
title_embs_df.to_csv(f'title_embs_df_{data_dir}.csv')

In [29]:
abstract_embs_df.to_csv(f'abstract_embs_df_{data_dir}.csv')